In [ ]:
import os
import random
import geopandas as gpd
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from django.contrib.gis.geos import GEOSGeometry

In [ ]:
geopackage = '/Users/marcellodebarrosfilho/code/geopackage/cassilandia.gpkg'

In [ ]:
setor_gdf = gpd.read_file(geopackage, layer='setores')

In [ ]:
quadra_gdf = gpd.read_file(geopackage, layer='quadras')

In [ ]:
lote_gdf = gpd.read_file(geopackage, layer='lotes')

In [ ]:
lote_gdf = lote_gdf.fillna('não informado')

In [ ]:
data = []
for idx, row in setor_gdf.iterrows():
    setor = Setor()
    setor.num_setor = row['setor']
    setor.geom = GEOSGeometry(f"{row['geometry']}")
    data.append(setor)

In [ ]:
Setor.objects.bulk_create(data)

In [ ]:
data = []
for idx, row in quadra_gdf.iterrows():
    quadra = Quadra()
    quadra.num_quadra = row['quadra']
    quadra.geom = GEOSGeometry(f"{row['geometry']}")
    setor = Setor.objects.filter(geom__contains=quadra.geom.point_on_surface).first()
    quadra.setor = setor
    data.append(quadra)

In [ ]:
Quadra.objects.bulk_create(data)

In [ ]:
SITUACAO_CHOICES = (
    ('a-ser-atendido', 'A ser atendido'),
    ('lote-cadastrado', 'Lote cadastrado'),
    ('lote-titulado', 'Lote titulado'),
)

In [ ]:
data = []
for idx, row in lote_gdf.iterrows():
    lote = Lote()
    lote.num_lote = row['lote']
    lote.geom = GEOSGeometry(f"{row['geometry']}")
    lote.situacao = random.choice(SITUACAO_CHOICES)[0]
    quadra = Quadra.objects.filter(geom__contains=lote.geom.point_on_surface).first()
    lote.quadra = quadra
    data.append(lote)

In [ ]:
Lote.objects.bulk_create(data)